In [ ]:
import copy, sys
import random, numpy as np

sys.path.append("../../")
from importlib import reload

import inferring.InferringDFA as InferringDFA
import utils.automata.DFA
import utils.advice_systems.SRS as SRS
import utils.runLearnLib.RunLearnLib as RunLearnLib
import utils.display.tables as Tables

reload(InferringDFA)
reload(RunLearnLib)
reload(utils.automata.DFA)
reload(SRS)
reload(Tables)

from inferring.InferringDFA import InferringDFA
from utils.runLearnLib.RunLearnLib import RunLearnLib
from utils.automata.DFA.DFA import DFA
from utils.advice_systems.SRS import SRS
from utils.display.tables import Tables

In [2]:
def run_learning_process(target, advice_system=None, check_consistency=False, equiv_query_fashion="BFS", debug=False):
    _dfa = copy.deepcopy(target)
    learn_dfa = InferringDFA(_dfa, 
                                advice_system, 
                                check_consistency=check_consistency, 
                                equiv_query_fashion=equiv_query_fashion,
                                debug=debug)
        
    dfa, cnt, cnt_ex = learn_dfa.run(counterexamples=True)
    return copy.deepcopy(dfa), cnt, len(cnt_ex) +1 

In [3]:
class Record:
    def __init__(self,
                 dfa_sz,
                 lstar,
                 lstar_as, 
                 target=None):
        self.dfa_sz = dfa_sz
        self.lstar = lstar
        self.lstar_as = lstar_as
        self.target = target

    def print_record(self):
        print(
            f"|conv| = {self.dfa_sz} eq = {self.lstar[1]}, eq_with_advice = {self.lstar_as[1]}"
        )
    def print_target(self):
        print(self.target.print_complete_description())

In [ ]:
max_number_of_states = 1000
number_of_iteration = 25
input_signs = ['a', 'b', 'c', 'd']

# Structure to aggregate the results.
results = []
i = 0

helper = RunLearnLib()
# Build the necessary part of the learnLib project.
helper.compileLearnLib()

while i < number_of_iteration:
    random.seed(i)
    i += 1
    # Create a random DFA with an idempotent letter and run the learning process to minimize it.
    # Exclude the degenerate cases (DFA of size smaller than 2).
    dfa = DFA()
    while dfa.Q < 2:
        d = DFA()
        d.create_random_indempotent_automaton(Q=random.randint(
            max_number_of_states // 2, max_number_of_states),
                                              input_signs=input_signs,
                                              letter=input_signs[0])
        dfa, _, _ = run_learning_process(target=copy.deepcopy(d))

    d.save_complete_description()

    # Learn conv_d without advice using LearnLib implementation of L*.
    lstar_output = helper.runLstar()
    # Learn conv_d with advice using LearnLib implementation of L*.
    lstar_as_output = helper.runLstarIdemWithAdvice()

    results.append(
        Record(dfa_sz=d.Q,
               lstar=lstar_output,
               lstar_as=lstar_as_output,
               target=d))

    print(f"iter {i} done")

In [8]:
results.sort(key=lambda x: (x.dfa_sz))

In [9]:
lstar_eqs = [r.lstar[1] for r in results]
lstar_as_eqs = [r.lstar_as[1] for r in results]

red = (
    (np.array(lstar_eqs) - np.array(lstar_as_eqs)) / np.array(lstar_eqs)) * 100
columns = [('Target language', 'DFA'), ('L*', 'EQ'), ('L* with advice', 'EQ'), ('Reduction', 'EQ')]
data = {
    columns[0]: [r.dfa_sz for r in results],
    columns[1]: lstar_eqs,
    columns[2]: lstar_as_eqs,
    columns[3]: [int(r) for r in red]
}

table_creator = Tables()
df = table_creator.create_basic_table(columns=columns, data=data)
display(df)

,Target language,L*,L* with advice,Reduction
,DFA,EQ,EQ,EQ
0,554,37,25,32%
1,568,51,36,29%
2,584,42,34,19%
3,592,54,35,35%
4,616,49,42,14%
5,620,48,47,2%
6,621,45,43,4%
7,632,48,36,25%
8,665,49,47,4%
